<a href="https://colab.research.google.com/github/HenryZhou1002/12306/blob/master/gan_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch

In [5]:
# Load PyTorch library
!pip3 install torch
!pip3 install torchvision
!pip install pillow==4.1.1
%reload_ext autoreload
%autoreload

    100% |████████████████████████████████| 5.7MB 6.7MB/s 
  Found existing installation: Pillow 5.4.1
    Uninstalling Pillow-5.4.1:
      Successfully uninstalled Pillow-5.4.1
  [PIL]
You must restart the runtime in order to use newly installed versions.


In [7]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jan 19 08:06:53 2019

@author: zhouhang
"""
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.autograd


device = torch.device('cuda')
# 进行数据预处理和迭代器的构建
im_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])    # 标准化
        ])

batch_size = 128

# 手写数字识别,image and labels
train_dataset = torchvision.datasets.MNIST(root='../mnist',
                                           train=True,
                                           transform = im_transforms,  # 使用了前面定义的迭代器,顺便做了数据的预处理
                                           download = True)

test_dataset = torchvision.datasets.MNIST(root='../mnist',
                                          train=False,
                                          transform=transforms.ToTensor())

# 加载数据 input pipline
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

# 定义网络:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        
        self.encoder = nn.Sequential(
                nn.Linear(28*28, 128),
                nn.ReLU(True),
                nn.Linear(128,64),
                nn.ReLU(True),
                nn.Linear(64,12),
                nn.ReLU(True),
                nn.Linear(12,3)    # 输出的code是三维的code
                )
        
        self.decoder = nn.Sequential(
                nn.Linear(3,12),
                nn.ReLU(True),
                nn.Linear(12,64),
                nn.ReLU(True),
                nn.Linear(64,128),
                nn.ReLU(True),
                nn.Linear(128,28*28),
                nn.Tanh()
                )
        
    def forward(self, x):
        encode = self.encoder(x)
        decode = self.decoder(encode)
        return encode, decode
    
net = autoencoder().cuda()
if torch.cuda.is_available():
  print('cuda available!!')
  net = net.cuda()
x = torch.autograd.Variable(torch.randn(10,28*28)).cuda()  # 随机生成一张图片,batch_size = 1
encode, _  = net(x)
#print(encode.shape, decode.shape)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.01)

def to_img(x):
    '''
    定义一个函数,将最后输出的结果,转换回图片
    '''
    x = 0.5 * (x +1.)
    x = x.clamp(0,1)
    x = x.view(x.shape[0],1,28,28)
    return x

# 开始训练自动编码器
for e in range(10):
    count = 0
    for im,_ in train_dataset:
        im = im.view(im.shape[0],-1).cuda()
        im = torch.autograd.Variable(im).cuda()
        
        # 向前传播
        _, output = net(im)
        loss = criterion(output,im)/im.shape[0]
        
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count+=1
        if (count%5000==0):
          print('count:{}'.format(count))
        
    
    if (e+1)%2 == 0:
        print('epoch: {}, Loss: {}'.format(e+1, loss.data))
        pic = to_img(output.data)
        
        # os.mkdir('./simple_autoencoder')
        torchvision.utils.save_image(pic,'./simple_autoencoder/image_{}.jpg'.format(e+1))            

cuda available!!
count:5000
count:10000
count:15000
count:20000
count:25000
count:30000
count:35000
count:40000
count:45000
count:50000
count:55000
count:60000
count:5000
count:10000
count:15000
count:20000
count:25000
count:30000
count:35000
count:40000
count:45000
count:50000
count:55000
count:60000
epoch: 2, Loss: 0.19224104285240173
count:5000
count:10000
count:15000
count:20000
count:25000
count:30000
count:35000
count:40000
count:45000
count:50000
count:55000
count:60000
count:5000
count:10000
count:15000
count:20000
count:25000
count:30000
count:35000
count:40000
count:45000
count:50000
count:55000
count:60000
epoch: 4, Loss: 0.19224104285240173
count:5000
count:10000
count:15000
count:20000
count:25000
count:30000
count:35000
count:40000
count:45000
count:50000
count:55000
count:60000
count:5000
count:10000
count:15000
count:20000
count:25000
count:30000
count:35000
count:40000
count:45000
count:50000
count:55000
count:60000
epoch: 6, Loss: 0.19224104285240173
count:5000
count: